In [106]:
import numpy as np
from scipy.stats import poisson, expon, uniform

In [107]:
## fill in your uni here

uni = 'jn2810'

## Background

Let's define the following terms for an insurance policy:

* A policy holder is the person who is purchasing insurance; the insurer is the party selling insurance
* The premium (P) is the amount that the policy holder pays every year to have insurance
* The deductible (D) is the amount of costs the policy holder is responsible for paying every year before insurance pays anything
* The annual maximum (A) is the maximum amount of money the policy holder will have to pay in a given year, anything above that is fully covered by the insurance company
* The co-insurance (C) is the percentage of costs the policy holder must pay on any costs above the deductible up until their annual maximum. Equivalently, the insurance company will be pay 0% of costs up to D, (1-C)% of the costs between D and A, and 100% of costs after D. 

For example, if I have auto collision insurance with a $1000$$ USD deductible, $25\%$$ coinsurance, and a $2000$$ USD annual maximum, and I get into accidents in the course of the year that cost $500$ USD to repair, I will have to pay the full $500$ USD since I have not met my deductible. 

If the accidents cost $4000$ USD to repair, I will have to pay $1750$ USD - 1000 from the deductible and $25\%$ of the $3000$ USD past the deductible - meaning the insurance company pays out 2250. 

If the accidents cost $8000$ USD to repair, I will have to pay $2000$, ($1000$ for the deductible, $1000$ from the next $4000$) and the insurance company will pay 6000. 

## Q1 (10 points)

The distribution of damages from an auto collision can be modelled with a Pareto Distribution with shape parameter $\alpha$, and scale parameter $\sigma$ which has the cdf:

$$F(x) = \begin{cases} 1 - (x/\sigma) ^{-\alpha} & x\geq1 \\
0 & x < 1
\end{cases} $$

Write a function sample_pareto(alpha, sigma, n) that generates n samples from a Pareto distribution using the inverse transform.

In [108]:
def sample_pareto(alpha, sigma, n):
    u = uniform.rvs(size = n)
    x = sigma / ((1 - u) ** (1 / alpha))
    return x


## Q2 (10 points)

The number of accidents that some drivers have in a given year can be described with an overdispersed Poisson Distribution. In this case, we generate $\lambda$ from an exponential distribution with expectation $\beta$, and then generate $X|\lambda \sim Poisson(\lambda)$ - i.e. every single data point has a different poisson parameter sampled from an exponential. We will say $X\sim ODP(\beta)$ to mean $X$ has such a distribution. 

Write a function sample_od_poisson(beta, n) that generates n samples from the overdispersed Poisson distribution. (Hint: use expon.rvs() with the appropriate scale parameter and size=n to generate a set of lambdas, then pass them poisson.rvs())

In [109]:
def sample_od_poisson(beta, n):
    lams = expon.rvs(scale = beta, size = n)
    X = poisson.rvs(mu = lams)
    return X


## Q3 (20 points)
Let $N$ be the number of accidents a driver has in a year; let $D$ be the amount of damages from those collisions.

There are three kinds of drivers:

* Risky drivers are 10% of the population. For risky drivers, $N$ follows a overdispersed Poisson with parameter $\beta=2$, and the damages from each accident follow a Pareto distribution with parameters $\alpha=2.5$, $\sigma=2000$.

* Normal drivers are 70% of the population. For normal drivers, $N$ follows a regular Poisson with parameter $\lambda=0.25$, and the damages from each accident follow a Pareto distribution with parameters $\alpha=4$, $\sigma=1000$.

* Cautious drivers are 20% of the population. For normal drivers, $N$ follows a regular Poisson with parameter $\lambda=0.1$, and the damages from each accident follow a Pareto distribution with parameters $\alpha=4$, $\sigma=1000$.

Write a function sample_driver(n) the number of accidents that a driver has in each of two years (N1 and N2), and the corresponding total damages for driver in each of those two years (D1) and (D2).

Hint: use a uniform to figure out what type of driver each person is, generate two poissons or overdispersed poissons depending on what kind of driver they are, generate two sets of the appropriate number/type of paretos for each person, and take the sum of each. repeat for each person


In [110]:
def sample_driver(n):
    N1, N2, D1, D2 = [], [], [], []
    W = uniform.rvs(size = n)
    for i in range(n):
        if W[i] < 0.1:
            a = sample_od_poisson(2, 1)
            b = sample_od_poisson(2, 1)
            c = sum(sample_pareto(2.5, 2000, a))
            d = sum(sample_pareto(2.5, 2000, a))



        elif W[i] < 0.8:
            a = poisson.rvs(0.25)
            b = poisson.rvs(0.25)
            c = sum(sample_pareto(4, 1000, a))
            d = sum(sample_pareto(4, 1000, a))
        else:
            a = poisson.rvs(0.1)
            b = poisson.rvs(0.1)
            c = sum(sample_pareto(4, 1000, a))
            d = sum(sample_pareto(4, 1000, a))
        
        N1.append(a)
        N2.append(b)
        D1.append(c)
        D2.append(d)

    
    return N1, N2, D1, D2



## Q4 (20 points)

Let's suppose that every policy has a deductible of $1000$ USD, $25\%$ co-insurance, and a $2000$ USD annual maximum.

Use simulation ($n=10,000$ samples) to estimate the premium that the insurance company should charge to break-even in the first year.

Hint: write a function that calculates the amount the insurance company needs to cover for the amount of damages a driver has incured. Calculate this for each driver in D1 and take the mean.

In [111]:
n = 10000

N1, N2, D1, D2 = sample_driver(n)

def payout(D):
    payout = []
    for i in range(len(D)):
        if D[i] <= 1000:
            x = 0
        elif D[i] <= 5000:
            x = 0.75 * (D[i] - 1000)
        else:
            x = D[i] - 2000
        payout.append(x)
    return np.mean(payout)

payout(D1)


591.8383592373127

## Q5 (20 points)

Calculate the expected payout from the insurance company (using the samples you have generated) in the second year for drivers who have $N=0, N=1, N=2, N=3$ and $N\geq 4$ accidents (i.e group ) in year one.

Hint: re-use the samples you generated in the last section and calculate the mean of D2 for all the drivers who have N1=0, etc.

In [112]:
e, f, g, h, j = [], [], [], [], []
for i in range(n):
    if N1[i] == 0:
        e.append(D2[i])
    elif N1[i] == 1:
        f.append(D2[i])
    elif N1[i] == 2:
        g.append(D2[i])
    elif N1[i] == 3:
        h.append(D2[i])
    elif N1[i] >= 4:
        j.append(D2[i])

payout(e), payout(f), payout(g), payout(h), payout(j)

(0.0,
 437.7495113568434,
 2538.427731727802,
 7232.024270312667,
 18430.529772989925)

## Q6 (20 points)

The insurance company will, in the second year, increase the premiums of anyone whose expected costs (to the insurance company) are greater than the break-even premium from part 4 to the level of their expected costs conditional on the number of accidents they had in year one, while everyone else will stay the same. How much profit do we expect the insurance company to make per customer in year two?

Hint: The new premium for each driver will be the greater of the value you calculated for them in the last section and the breakeven from part 4. Take the difference between the mean of this and the breakeven from part 4.

In [113]:
ec = [payout(e), payout(f), payout(g), payout(h), payout(j)]


In [114]:
def new_Pre():
    a = []
    for i in N1:

    return np.mean(a)

print(new_Pre() - payout(D1))

SyntaxError: incomplete input (3108110377.py, line 3)